In [1]:
import pandas as pd
import numpy as np

In [2]:
## Reading dataset
df = pd.read_csv('/Users/joaopedro/Documents/MSBA/Interview Prep/Data Interview/product_sql/facebook_connections/friends_connections.csv')
df.head()
print(len(df))

4841


#### FRIENDS_CONNECTIONS 

Facebook’s analytics team wants to understand how users stay connected among friends on their platform. 

The team believes that understanding patterns could help improve an algorithm that matches potential friends. 

**1. Return a list of users who blocked another user after connecting for at least 90 days. Show the user_id and receiver_id.**

- Self join table, joining conection and block tables on the condition that datediff(block, connect) >= 90

In [3]:
## Creating datetime column
from datetime import datetime

df['dates'] = pd.to_datetime(df['dates'])
df.head()

,dates,user_id,receiver_id,action
0,2020-01-30,100,246,Sent
1,2020-01-01,100,895,Received
2,2020-05-03,100,895,Connected
3,2020-02-06,101,678,Sent
4,2020-04-14,101,678,Connected


In [4]:
## One table with connected, and one with blocked
connected = df[df['action'] == 'Sent']
blocked = df[df['action'] == 'Blocked']

print(len(blocked))
len(connected)

287


1339

In [5]:
blocked.head()

,dates,user_id,receiver_id,action
21,2020-03-28,103,481,Blocked
28,2020-06-24,104,744,Blocked
31,2020-06-06,104,608,Blocked
34,2020-05-19,105,292,Blocked
42,2020-06-20,107,415,Blocked


In [6]:
## Join Tables
joined = blocked.merge(connected , how = 'inner', on = ['user_id', 'receiver_id'])

## Filter 90 days
# joined[joined['action_y'] - joined['action_x'] > 0]
len(joined)

136

In [7]:
## Create date difference column
joined['dif'] = (joined['dates_x'] - joined['dates_y']).dt.days
joined

,dates_x,user_id,receiver_id,action_x,dates_y,action_y,dif
0,2020-03-28,103,481,Blocked,2020-01-17,Sent,71
1,2020-06-24,104,744,Blocked,2020-03-02,Sent,114
2,2020-05-19,105,292,Blocked,2020-02-12,Sent,97
3,2020-05-25,121,263,Blocked,2020-01-26,Sent,120
4,2020-06-20,130,740,Blocked,2020-04-24,Sent,57
...,...,...,...,...,...,...,...
131,2020-06-20,978,862,Blocked,2020-02-16,Sent,125
132,2020-03-13,983,208,Blocked,2020-01-23,Sent,50
133,2020-06-01,986,751,Blocked,2020-01-17,Sent,136
134,2020-03-19,991,436,Blocked,2020-01-17,Sent,62


In [8]:
## Filter Difference > 90
len(joined[joined['dif'] >= 90])

70

In [9]:
## Show Answer
joined

,dates_x,user_id,receiver_id,action_x,dates_y,action_y,dif
0,2020-03-28,103,481,Blocked,2020-01-17,Sent,71
1,2020-06-24,104,744,Blocked,2020-03-02,Sent,114
2,2020-05-19,105,292,Blocked,2020-02-12,Sent,97
3,2020-05-25,121,263,Blocked,2020-01-26,Sent,120
4,2020-06-20,130,740,Blocked,2020-04-24,Sent,57
...,...,...,...,...,...,...,...
131,2020-06-20,978,862,Blocked,2020-02-16,Sent,125
132,2020-03-13,983,208,Blocked,2020-01-23,Sent,50
133,2020-06-01,986,751,Blocked,2020-01-17,Sent,136
134,2020-03-19,991,436,Blocked,2020-01-17,Sent,62


**2. For each user, what is the proportion of each action?** 

Note that the receiver_id can appear in multiple actions per user, only regard the latest status when calculating the distribution. 

-- group by user_id and action, counting the number of each action

In [10]:
## Create count column
df['pos'] = df.sort_values(['dates'], ascending=[False]) \
                 .groupby(['user_id','receiver_id']) \
                 .cumcount()+1
## Filter just the latest status
ft = df[df['pos'] == 1]
ft

,dates,user_id,receiver_id,action,pos
0,2020-01-30,100,246,Sent,1
2,2020-05-03,100,895,Connected,1
4,2020-04-14,101,678,Connected,1
5,2020-01-03,101,790,Sent,1
7,2020-06-11,101,313,Connected,1
...,...,...,...,...,...
4831,2020-04-29,997,185,Connected,1
4833,2020-05-18,997,916,Connected,1
4835,2020-05-09,998,651,Connected,1
4838,2020-05-02,998,768,Blocked,1


In [11]:
# Num actions
n_action = pd.pivot_table(ft, index= ['user_id','action'], aggfunc = 'count')
n_action.head()

dates  pos  receiver_id
user_id action                            
100     Connected      1    1            1
        Sent           1    1            1
101     Connected      2    2            2
        Sent           2    2            2
102     Connected      5    5            5

In [12]:
# total Actions
ft['total'] = ft.sort_values(['dates'], ascending=[False]) \
                .groupby('user_id') \
                .cumcount()+1
ft

/var/folders/vd/d1gk2l750jl53z2v384n4vvc0000gn/T/ipykernel_5913/3884630917.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ft['total'] = ft.sort_values(['dates'], ascending=[False]) \


,dates,user_id,receiver_id,action,pos,total
0,2020-01-30,100,246,Sent,1,2
2,2020-05-03,100,895,Connected,1,1
4,2020-04-14,101,678,Connected,1,2
5,2020-01-03,101,790,Sent,1,4
7,2020-06-11,101,313,Connected,1,1
...,...,...,...,...,...,...
4831,2020-04-29,997,185,Connected,1,2
4833,2020-05-18,997,916,Connected,1,1
4835,2020-05-09,998,651,Connected,1,2
4838,2020-05-02,998,768,Blocked,1,3


In [20]:
f_table = pd.pivot_table(df, index= ['user_id','action'], aggfunc = 'sum')['pos']/pd.pivot_table(df, index= ['user_id'], aggfunc = 'sum')['pos']
f_table

user_id  action   
100      Connected    0.250000
         Received     0.500000
         Sent         0.250000
101      Connected    0.250000
         Sent         0.750000
                        ...   
997      Sent         0.333333
998      Blocked      0.083333
         Connected    0.333333
         Received     0.333333
         Sent         0.250000
Name: pos, Length: 2529, dtype: float64